# 지니 음원 차트 크롤링

## 월별

In [187]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time

driver = webdriver.Chrome('./chromedriver_win32/chromedriver.exe')

song_data = []

for month in range(1, 13):
    rank = 1
    for page in range(1,3):
        url = f"https://www.genie.co.kr/chart/top200?ditc=M&ymd=2020{month:02d}01&hh=15&rtm=N&pg={page}"
        driver.get(url)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
    
        songs = soup.select('table > tbody > tr')
        i = 0
        for song in songs:
            title = song.select('a.title')[0].text.strip()
            singer = song.select('a.artist')[0].text

            select = driver.find_elements_by_css_selector('a.btn-info')[i].click()
            #time.sleep(1)

            genre = driver.find_elements_by_css_selector('span.value')[2].text
            #time.sleep(1)
            driver.back()

            i += 1

            song_data.append(['2020', month, rank, title, singer, genre])
            rank += 1

In [188]:
# 데이터 프레임으로 만든 후 엑셀파일로 저장
import pandas as pd

columns = ['년도', '월', '순위', '타이틀', '가수', '장르']
pd_data = pd.DataFrame(song_data, columns = columns)

pd_data.to_excel('./data/2020_month.xlsx', index=False)

## 연도별

In [91]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time

driver = webdriver.Chrome('./chromedriver_win32/chromedriver.exe')

song_data = []

rank = 1
for page in range(1,3):
    url = f"https://www.genie.co.kr/chart/musicHistory?year=2020&category=0&pg={page}"
    driver.get(url)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    songs = soup.select('table > tbody > tr')

    i = 0
    for song in songs:
        title = song.select('a.title')[0].text.split('\n')[-1].strip()
        singer = song.select('a.artist')[0].text

        select = driver.find_elements_by_css_selector('a.btn-info')[i].click()
        #time.sleep(1)

        genre = driver.find_elements_by_css_selector('span.value')[2].text
        #time.sleep(1)
        driver.back()

        i += 1

        song_data.append(['2020', rank, title, singer, genre])
        rank += 1

In [93]:
# 데이터 프레임으로 만든 후 엑셀파일로 저장
import pandas as pd

columns = ['년도', '순위', '타이틀', '가수', '장르']
pd_data = pd.DataFrame(song_data, columns = columns)

pd_data.to_excel('./data/2020_year.xlsx', index=False)

# 멜론 음원 차트 크롤링

## 연도별

In [407]:
from selenium import webdriver
from bs4 import BeautifulSoup
import re

# 웹 드라이버 실행
driver1 = webdriver.Chrome('./chromedriver_win32/chromedriver.exe')
driver2 = webdriver.Chrome('./chromedriver_win32/chromedriver.exe')

url = "https://www.melon.com/chart/index.htm"
driver1.get(url)

song_data = []
year = 2020

# 차트 파인더 조건 선택
driver1.find_elements_by_css_selector('#gnb_menu > ul:nth-child(1) > li.nth1.on > div > div > button > span')[0].click()  # 차트 파인더 페이지로 이동
time.sleep(1)
driver1.find_elements_by_css_selector('#d_chart_search > div > h4.tab03 > a')[0].click()  # 차트 선택
time.sleep(1)
for i in range(1,3):
    driver1.find_elements_by_css_selector(f'#d_chart_search > div > div > div.box_chic.nth1.view > div.list_value > ul > li:nth-child({i})')[0].click()  # 연대 선택
    time.sleep(1)
    for j in range(1,4):
        try:
            driver1.find_elements_by_css_selector(f'#d_chart_search > div > div > div.box_chic.nth2.view > div.list_value > ul > li:nth-child({j})')[0].click()  # 연도 선택
            time.sleep(1)
        except IndexError :
            continue
        driver1.find_elements_by_css_selector('#d_chart_search > div > div > div.box_chic.last.view > div.list_value > ul > li:nth-child(1)')[0].click()  # 장르/스타일 선택
        time.sleep(1)
        driver1.find_elements_by_css_selector('#d_srch_form > div.wrap_btn_serch > button > span > span')[0].click()  # 검색 버튼 클릭
        time.sleep(1)

        html = driver1.page_source
        soup = BeautifulSoup(html, 'html.parser')
        songs = soup.select('table > tbody > tr')
        rank = 1

        for song in songs:
            title = song.select('div.rank01 > span > strong > a')[0].text
            singer = song.select('div.rank02 > a')[0].text

            href = song.select('a.btn_icon_detail')[0].get('href')  # 곡 정보 버튼에서 href 속성값 가져오기
            songId = re.findall("\d+", href)[0]  # songId 추출

            url = f'https://www.melon.com/song/detail.htm?songId={songId}'  # 곡 정보 페이지 이동
            driver2.get(url)

            genre = driver2.find_elements_by_css_selector('div.meta > dl > dd:nth-child(6)')[0].text

            song_data.append([year, rank, title, singer, genre])
            rank += 1
        year -= 1

In [408]:
# 데이터 프레임으로 만든 후 엑셀파일로 저장
import pandas as pd

columns = ['년도', '순위', '타이틀', '가수', '장르']
pd_data = pd.DataFrame(song_data, columns = columns)

pd_data.to_excel('./data/melon_year.xlsx', index=False)